In [48]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from pprint import pprint
import openweathermapy.core as owm
from citipy import citipy
import random as random

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)
api_key

'6f4ceb243793f6a90aeed94e4f17b8a4'

## Generate Cities list

In [74]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=10) #size 1500
lngs = np.random.uniform(low=-180.000, high=180.000, size=10) #size 1500
#zip in to a list of tuples
lat_lngs = zip(lats, lngs)

#to print zipped object
#for i in lat_lngs:
#    print(i)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


10

## Perform API Calls

In [83]:
#save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial" #use imperial to get farenheit

#build partial string query URL
query_url = f"{url}appid={api_key}&units={units}&q="
query_url

#initialize lists to hold temp, humidity clouds and wind
temp = []
humidity= []
clouds = []
wind = []
city_not_found=[]
valid_cities=[]

for i in range(len(cities)):
    response = requests.get(query_url+cities[i]).json()
    
    if response["cod"] == "404":
        print(f"city number {i}, {cities[i]} is not found")
        city_not_found.append(cities[i])
    else:
        print(f"Getting data for city number {i}, {cities[i]}")
        valid_cities.append(cities[i])
        temp.append(response["main"]["temp"])
        humidity.append(response["main"]["humidity"])
        clouds.append(response["clouds"]["all"])
        wind.append(response["wind"]["speed"])

print(f"cities list {valid_cities}, len {len(valid_cities)}")  
print(f"temp list {temp}, len {len(temp)}")
print(f"humidity list {humidity}, len {len(humidity)}")
print(f"clouds list {clouds}, len {len(clouds)}")
print(f"wind list {wind}, len {len(wind)}")

city_not_found


Getting data for city number 0, ushuaia
Getting data for city number 1, ajdabiya
Getting data for city number 2, butaritari
Getting data for city number 3, itarema
Getting data for city number 4, saint george
Getting data for city number 5, castro
city number 6, marcona is not found
Getting data for city number 7, provideniya
Getting data for city number 8, yenagoa
Getting data for city number 9, chokurdakh
cities list ['ushuaia', 'ajdabiya', 'butaritari', 'itarema', 'saint george', 'castro', 'provideniya', 'yenagoa', 'chokurdakh'], len 9
temp list [52, 55.02, 85.08, 80.17, 38.37, 55.4, -8.46, 80.11, -21.12], len 9
humidity list [66, 64, 79, 75, 47, 87, 91, 66, 99], len 9
clouds list [75, 99, 100, 8, 20, 90, 100, 100, 28], len 9
wind list [3.36, 19.35, 19.01, 6.13, 9.17, 12.75, 3.91, 6.44, 2.77], len 9


['marcona']

## Create master table

In [85]:
#create dictionary of cities, temp, humidity, clouds, and wind
d = {"Cities": cities, "Temperature": temp, "Humidity": humidity, "Clouds": clouds, "Wind": wind}

#convert dictionary to dataframe
df = pd.DataFrame(d)

ValueError: arrays must all be same length

In [12]:
#Get sample response for reference

response=requests.get("http://api.openweathermap.org/data/2.5/weather?appid=6f4ceb243793f6a90aeed94e4f17b8a4&units=imperial&q=paris").json()
pprint(response)


{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 48.86, 'lon': 2.35},
 'dt': 1578338058,
 'id': 2988507,
 'main': {'feels_like': 30.51,
          'humidity': 75,
          'pressure': 1025,
          'temp': 41,
          'temp_max': 42.01,
          'temp_min': 39.99},
 'name': 'Paris',
 'sys': {'country': 'FR',
         'id': 6540,
         'sunrise': 1578296602,
         'sunset': 1578326913,
         'type': 1},
 'timezone': 3600,
 'visibility': 10000,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 230, 'speed': 12.75}}
